In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
import random

random.seed(0)
weight_decay = 0.0005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'fashion-mnist': ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag',
                          'ankle-boot'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma, gamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print('training class:',single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    print('Start of the proposed transformation')
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print('End of the proposed transformation')
    batch_size = 128
    print('Start of the training')
    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=batch_size, epochs= 5
            )
    print('End of the training')

    #Estimating the parameters
    def calc_approx_alpha_sum(observations):
        N = observations.shape[0]
        K = observations.shape[1]
        f = np.mean(observations, axis=0)
        log_obs = (np.nan_to_num(np.log(np.clip(observations,10**(-10),None)), copy=True, nan=10, posinf=10**10, neginf=-(10**10)))

        return (N * (K - 1) * (-psi(1))) / (
                N * sum(f * np.log(f)) + sum([np.log(sum(observations[j][:K-1])) for j in range(N)]) - sum([sum(f*log_obs[j]) for j in range(N)]) )
    def calcul_sum_alpha_beta(observations):
      N = observations.shape[0]
      K = observations.shape[1]
      f = np.mean(np.mean(observations, axis=0))
      return (N * (-psi(1))) / (
                2*N*(f * np.log(f)) -f* sum([np.log(sum(observations[j][:K-1])) for j in range(N)]) - f*sum([np.log(observations)[j][K-1] for j in range(N)]) )

    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x

    def fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,y, max_iter=100):
        log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10)).mean(axis=0)
        N = y.shape[0]
        K = y.shape[1]
        alpha_old = alpha_0
        beta_old = beta_0
        vect_alpha_old = vect_alpha_0

        for iterr in range(max_iter):
          #calculate alpha
          aph = psi(alpha_old + beta_old) + (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)])
          aph = np.nan_to_num(aph, copy=True, nan=10**3, posinf=10**10, neginf=-(10**10))
          alpha_new = inv_psi(np.clip(aph,None,7), iters=5)
          alpha_new = np.clip(alpha_new,10**(-10),None)

          #calculate beta
          bet = psi(alpha_old + beta_old) + log_y[K-1]
          bet = np.nan_to_num(bet, copy=True, nan=0.1, posinf=10**10, neginf=-(10**10))
          beta_new = inv_psi(np.clip(bet,None,7), iters=5)
          beta_new = np.clip(beta_new,10**(-10),None)

          #calculate the vector alpha
          vect_alpha_new = np.asarray([inv_psi(psi(sum(vect_alpha_old)) + log_y[i] - (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)]) ,iters=5) for i in range(K)])
          vect_alpha_new = np.nan_to_num((np.clip(vect_alpha_new,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10))
          if np.sqrt(np.sum((vect_alpha_old - vect_alpha_new) ** 2)) < 1e-3 and np.sqrt((alpha_new - alpha_old) ** 2) < 1e-3 and np.sqrt((beta_new - beta_old) ** 2) < 1e-3:
            break
          #update the parameters
          alpha_old = alpha_new
          beta_old = beta_new
          vect_alpha_old = vect_alpha_new
        return vect_alpha_new, alpha_new, beta_new

    #Normality Score
    def beta_louiville_normality_score(vect_alpha,alpha,beta,y):
      N=y.shape[0]
      K=y.shape[1]
      log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10))
      scores = np.asarray([sum((vect_alpha-1)*log_y[j]) + (beta-1)*log_y[j][K-1] + (alpha-sum(vect_alpha))*np.log(sum(y[j][:k-1]))  for j in range(N)]) #+ np.log(gamma(sum(vect_alpha))) + np.log(gamma(alpha+beta)) - np.log(gamma(alpha)) - np.log(gamma(beta)) - sum([np.log(gamma(vect_alpha[i])) for i in range(K)])
      scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
      return scores

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=128)
        alpha_sum_approx = np.abs(calc_approx_alpha_sum(observed_dirichlet))
        alpha_sum_approx = np.nan_to_num(alpha_sum_approx, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        vect_alpha_0 = np.clip(observed_dirichlet.mean(axis=0) * alpha_sum_approx,10**(-7),10**7)
        alpha_0 = 0.01
        beta_0 = 0.01

        mle_vect_alpha_t,mle_alpha_t,mle_beta_t = fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,observed_dirichlet,max_iter=10)
        mle_vect_alpha_t=np.nan_to_num(mle_vect_alpha_t, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        mle_alpha_t= np.nan_to_num(mle_alpha_t, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        mle_beta_t= np.nan_to_num(mle_beta_t, copy=True, nan=1, posinf=10**10, neginf=-10**10)

        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=128)
        scores += beta_louiville_normality_score(mle_vect_alpha_t,mle_alpha_t,mle_beta_t, x_test_p)
        scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 1
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128
N_EPOCHS = 5

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [
        (load_mnist, 'mnist', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
training class: 0
11501568/11490434 [==============================] - 59s 5us/step
Wide Residual Network-16-8 created.
Start of the proposed transformation
End of the proposed transformation
Start of the training
Epoch 1/5


198/564 [=========>....................] - ETA: 1:54:35 - loss: 8.8464 - acc: 0.00 - ETA: 1:20:09 - loss: 8.7473 - acc: 0.02 - ETA: 1:19:15 - loss: 8.6488 - acc: 0.03 - ETA: 1:17:43 - loss: 8.5524 - acc: 0.04 - ETA: 1:17:18 - loss: 8.4580 - acc: 0.04 - ETA: 1:16:37 - loss: 8.3651 - acc: 0.05 - ETA: 1:16:15 - loss: 8.2738 - acc: 0.05 - ETA: 1:14:35 - loss: 8.1845 - acc: 0.05 - ETA: 1:14:55 - loss: 8.0974 - acc: 0.06 - ETA: 1:15:25 - loss: 8.0127 - acc: 0.06 - ETA: 1:15:11 - loss: 7.9299 - acc: 0.06 - ETA: 1:15:44 - loss: 7.8489 - acc: 0.07 - ETA: 1:15:54 - loss: 7.7705 - acc: 0.07 - ETA: 1:15:43 - loss: 7.6942 - acc: 0.07 - ETA: 1:15:40 - loss: 7.6204 - acc: 0.08 - ETA: 1:15:52 - loss: 7.5486 - acc: 0.08 - ETA: 1:15:51 - loss: 7.4798 - acc: 0.09 - ETA: 1:15:25 - loss: 7.4128 - acc: 0.08 - ETA: 1:14:50 - loss: 7.3477 - acc: 0.08 - ETA: 1:14:55 - loss: 7.2841 - acc: 0.09 - ETA: 1:15:04 - loss: 7.2216 - acc: 0.09 - ETA: 1:14:53 - loss: 7.1615 - acc: 0.09 - ETA: 1:14:39 - loss: 7.1026 - acc

402/564 [====================>.........] - ETA: 49:56 - loss: 3.3164 - acc: 0.30 - ETA: 49:48 - loss: 3.3084 - acc: 0.30 - ETA: 49:39 - loss: 3.3002 - acc: 0.30 - ETA: 49:29 - loss: 3.2920 - acc: 0.30 - ETA: 49:22 - loss: 3.2844 - acc: 0.30 - ETA: 49:17 - loss: 3.2763 - acc: 0.30 - ETA: 49:09 - loss: 3.2688 - acc: 0.30 - ETA: 48:59 - loss: 3.2612 - acc: 0.30 - ETA: 48:53 - loss: 3.2541 - acc: 0.30 - ETA: 48:45 - loss: 3.2465 - acc: 0.30 - ETA: 48:37 - loss: 3.2387 - acc: 0.30 - ETA: 48:29 - loss: 3.2310 - acc: 0.31 - ETA: 48:20 - loss: 3.2235 - acc: 0.31 - ETA: 48:12 - loss: 3.2161 - acc: 0.31 - ETA: 48:04 - loss: 3.2086 - acc: 0.31 - ETA: 47:55 - loss: 3.2013 - acc: 0.31 - ETA: 47:48 - loss: 3.1943 - acc: 0.31 - ETA: 47:40 - loss: 3.1872 - acc: 0.31 - ETA: 47:31 - loss: 3.1797 - acc: 0.31 - ETA: 47:22 - loss: 3.1724 - acc: 0.31 - ETA: 47:13 - loss: 3.1652 - acc: 0.31 - ETA: 47:06 - loss: 3.1584 - acc: 0.31 - ETA: 46:58 - loss: 3.1510 - acc: 0.32 - ETA: 46:50 - loss: 3.1442 - acc: 0.32

564/564 [==============================] - ETA: 21:59 - loss: 2.3596 - acc: 0.45 - ETA: 21:50 - loss: 2.3568 - acc: 0.45 - ETA: 21:42 - loss: 2.3543 - acc: 0.45 - ETA: 21:34 - loss: 2.3516 - acc: 0.45 - ETA: 21:26 - loss: 2.3487 - acc: 0.46 - ETA: 21:18 - loss: 2.3462 - acc: 0.46 - ETA: 21:09 - loss: 2.3436 - acc: 0.46 - ETA: 21:01 - loss: 2.3408 - acc: 0.46 - ETA: 20:53 - loss: 2.3380 - acc: 0.46 - ETA: 20:45 - loss: 2.3355 - acc: 0.46 - ETA: 20:37 - loss: 2.3329 - acc: 0.46 - ETA: 20:29 - loss: 2.3303 - acc: 0.46 - ETA: 20:20 - loss: 2.3277 - acc: 0.46 - ETA: 20:12 - loss: 2.3250 - acc: 0.46 - ETA: 20:04 - loss: 2.3226 - acc: 0.46 - ETA: 19:56 - loss: 2.3201 - acc: 0.46 - ETA: 19:47 - loss: 2.3175 - acc: 0.46 - ETA: 19:39 - loss: 2.3150 - acc: 0.46 - ETA: 19:31 - loss: 2.3124 - acc: 0.46 - ETA: 19:23 - loss: 2.3098 - acc: 0.46 - ETA: 19:15 - loss: 2.3070 - acc: 0.46 - ETA: 19:07 - loss: 2.3043 - acc: 0.47 - ETA: 18:59 - loss: 2.3017 - acc: 0.47 - ETA: 18:50 - loss: 2.2993 - acc: 0.47

197/564 [=========>....................] - ETA: 1:23:25 - loss: 1.0234 - acc: 0.77 - ETA: 1:29:46 - loss: 1.1092 - acc: 0.75 - ETA: 1:29:43 - loss: 1.1225 - acc: 0.75 - ETA: 1:30:45 - loss: 1.1510 - acc: 0.75 - ETA: 1:32:34 - loss: 1.1603 - acc: 0.74 - ETA: 1:29:38 - loss: 1.1568 - acc: 0.75 - ETA: 1:25:16 - loss: 1.1910 - acc: 0.73 - ETA: 1:24:00 - loss: 1.1957 - acc: 0.73 - ETA: 1:25:22 - loss: 1.1753 - acc: 0.74 - ETA: 1:24:35 - loss: 1.1698 - acc: 0.74 - ETA: 1:25:28 - loss: 1.1565 - acc: 0.74 - ETA: 1:26:21 - loss: 1.1467 - acc: 0.75 - ETA: 1:26:01 - loss: 1.1449 - acc: 0.75 - ETA: 1:25:50 - loss: 1.1416 - acc: 0.75 - ETA: 1:26:16 - loss: 1.1362 - acc: 0.76 - ETA: 1:26:01 - loss: 1.1338 - acc: 0.75 - ETA: 1:25:07 - loss: 1.1302 - acc: 0.76 - ETA: 1:25:16 - loss: 1.1245 - acc: 0.76 - ETA: 1:24:49 - loss: 1.1188 - acc: 0.76 - ETA: 1:25:02 - loss: 1.1142 - acc: 0.76 - ETA: 1:24:48 - loss: 1.1099 - acc: 0.76 - ETA: 1:24:07 - loss: 1.1060 - acc: 0.77 - ETA: 1:23:21 - loss: 1.1024 - acc

...

408/551 [=====================>........] - ETA: 31:37 - loss: 0.1786 - acc: 0.99 - ETA: 31:30 - loss: 0.1785 - acc: 0.99 - ETA: 31:25 - loss: 0.1784 - acc: 0.99 - ETA: 31:20 - loss: 0.1783 - acc: 0.99 - ETA: 31:14 - loss: 0.1782 - acc: 0.99 - ETA: 31:08 - loss: 0.1781 - acc: 0.99 - ETA: 31:03 - loss: 0.1780 - acc: 0.99 - ETA: 30:57 - loss: 0.1779 - acc: 0.99 - ETA: 30:51 - loss: 0.1778 - acc: 0.99 - ETA: 30:47 - loss: 0.1778 - acc: 0.99 - ETA: 30:41 - loss: 0.1776 - acc: 0.99 - ETA: 30:35 - loss: 0.1776 - acc: 0.99 - ETA: 30:29 - loss: 0.1775 - acc: 0.99 - ETA: 30:23 - loss: 0.1774 - acc: 0.99 - ETA: 30:17 - loss: 0.1776 - acc: 0.99 - ETA: 30:12 - loss: 0.1776 - acc: 0.99 - ETA: 30:07 - loss: 0.1776 - acc: 0.99 - ETA: 30:01 - loss: 0.1776 - acc: 0.99 - ETA: 29:56 - loss: 0.1775 - acc: 0.99 - ETA: 29:50 - loss: 0.1774 - acc: 0.99 - ETA: 29:44 - loss: 0.1773 - acc: 0.99 - ETA: 29:39 - loss: 0.1772 - acc: 0.99 - ETA: 29:33 - loss: 0.1772 - acc: 0.99 - ETA: 29:28 - loss: 0.1772 - acc: 0.99

551/551 [==============================] - ETA: 13:07 - loss: 0.1745 - acc: 0.99 - ETA: 13:01 - loss: 0.1745 - acc: 0.99 - ETA: 12:55 - loss: 0.1745 - acc: 0.99 - ETA: 12:50 - loss: 0.1744 - acc: 0.99 - ETA: 12:44 - loss: 0.1744 - acc: 0.99 - ETA: 12:39 - loss: 0.1743 - acc: 0.99 - ETA: 12:33 - loss: 0.1743 - acc: 0.99 - ETA: 12:27 - loss: 0.1743 - acc: 0.99 - ETA: 12:21 - loss: 0.1742 - acc: 0.99 - ETA: 12:16 - loss: 0.1742 - acc: 0.99 - ETA: 12:10 - loss: 0.1742 - acc: 0.99 - ETA: 12:04 - loss: 0.1742 - acc: 0.99 - ETA: 11:59 - loss: 0.1741 - acc: 0.99 - ETA: 11:53 - loss: 0.1741 - acc: 0.99 - ETA: 11:47 - loss: 0.1741 - acc: 0.99 - ETA: 11:42 - loss: 0.1741 - acc: 0.99 - ETA: 11:36 - loss: 0.1741 - acc: 0.99 - ETA: 11:31 - loss: 0.1741 - acc: 0.99 - ETA: 11:25 - loss: 0.1740 - acc: 0.99 - ETA: 11:20 - loss: 0.1740 - acc: 0.99 - ETA: 11:14 - loss: 0.1739 - acc: 0.99 - ETA: 11:09 - loss: 0.1739 - acc: 0.99 - ETA: 11:03 - loss: 0.1739 - acc: 0.99 - ETA: 10:58 - loss: 0.1739 - acc: 0.99

204/551 [==========>...................] - ETA: 56:43 - loss: 0.1691 - acc: 1.00 - ETA: 51:00 - loss: 0.2261 - acc: 0.98 - ETA: 50:50 - loss: 0.2070 - acc: 0.98 - ETA: 50:39 - loss: 0.1983 - acc: 0.99 - ETA: 49:57 - loss: 0.1984 - acc: 0.99 - ETA: 49:33 - loss: 0.1972 - acc: 0.99 - ETA: 49:00 - loss: 0.1966 - acc: 0.99 - ETA: 48:56 - loss: 0.1953 - acc: 0.99 - ETA: 49:07 - loss: 0.1985 - acc: 0.99 - ETA: 48:57 - loss: 0.1959 - acc: 0.99 - ETA: 49:01 - loss: 0.1970 - acc: 0.99 - ETA: 48:58 - loss: 0.1999 - acc: 0.99 - ETA: 49:00 - loss: 0.1984 - acc: 0.99 - ETA: 49:06 - loss: 0.1992 - acc: 0.99 - ETA: 49:13 - loss: 0.1993 - acc: 0.99 - ETA: 49:25 - loss: 0.1997 - acc: 0.99 - ETA: 49:41 - loss: 0.1986 - acc: 0.99 - ETA: 49:47 - loss: 0.1976 - acc: 0.99 - ETA: 49:37 - loss: 0.1971 - acc: 0.99 - ETA: 49:26 - loss: 0.1976 - acc: 0.99 - ETA: 49:30 - loss: 0.1970 - acc: 0.99 - ETA: 49:20 - loss: 0.1966 - acc: 0.99 - ETA: 49:03 - loss: 0.1960 - acc: 0.99 - ETA: 48:55 - loss: 0.1961 - acc: 0.99

408/551 [=====================>........] - ETA: 30:44 - loss: 0.1814 - acc: 0.99 - ETA: 30:35 - loss: 0.1815 - acc: 0.99 - ETA: 30:26 - loss: 0.1814 - acc: 0.99 - ETA: 30:18 - loss: 0.1815 - acc: 0.99 - ETA: 30:09 - loss: 0.1815 - acc: 0.99 - ETA: 30:01 - loss: 0.1815 - acc: 0.99 - ETA: 29:52 - loss: 0.1814 - acc: 0.99 - ETA: 29:44 - loss: 0.1814 - acc: 0.99 - ETA: 29:36 - loss: 0.1814 - acc: 0.99 - ETA: 29:27 - loss: 0.1815 - acc: 0.99 - ETA: 29:19 - loss: 0.1814 - acc: 0.99 - ETA: 29:11 - loss: 0.1814 - acc: 0.99 - ETA: 29:03 - loss: 0.1815 - acc: 0.99 - ETA: 28:55 - loss: 0.1814 - acc: 0.99 - ETA: 28:47 - loss: 0.1813 - acc: 0.99 - ETA: 28:39 - loss: 0.1812 - acc: 0.99 - ETA: 28:31 - loss: 0.1811 - acc: 0.99 - ETA: 28:23 - loss: 0.1811 - acc: 0.99 - ETA: 28:16 - loss: 0.1810 - acc: 0.99 - ETA: 28:08 - loss: 0.1810 - acc: 0.99 - ETA: 28:00 - loss: 0.1809 - acc: 0.99 - ETA: 27:52 - loss: 0.1808 - acc: 0.99 - ETA: 27:45 - loss: 0.1807 - acc: 0.99 - ETA: 27:37 - loss: 0.1806 - acc: 0.99

551/551 [==============================] - ETA: 10:15 - loss: 0.1766 - acc: 0.99 - ETA: 10:10 - loss: 0.1765 - acc: 0.99 - ETA: 10:05 - loss: 0.1764 - acc: 0.99 - ETA: 10:01 - loss: 0.1764 - acc: 0.99 - ETA: 9:56 - loss: 0.1763 - acc: 0.9952 - ETA: 9:51 - loss: 0.1763 - acc: 0.995 - ETA: 9:47 - loss: 0.1762 - acc: 0.995 - ETA: 9:42 - loss: 0.1762 - acc: 0.995 - ETA: 9:37 - loss: 0.1761 - acc: 0.995 - ETA: 9:33 - loss: 0.1762 - acc: 0.995 - ETA: 9:28 - loss: 0.1761 - acc: 0.995 - ETA: 9:24 - loss: 0.1761 - acc: 0.995 - ETA: 9:19 - loss: 0.1760 - acc: 0.995 - ETA: 9:14 - loss: 0.1759 - acc: 0.995 - ETA: 9:10 - loss: 0.1759 - acc: 0.995 - ETA: 9:05 - loss: 0.1758 - acc: 0.995 - ETA: 9:00 - loss: 0.1759 - acc: 0.995 - ETA: 8:56 - loss: 0.1759 - acc: 0.995 - ETA: 8:51 - loss: 0.1758 - acc: 0.995 - ETA: 8:47 - loss: 0.1757 - acc: 0.995 - ETA: 8:42 - loss: 0.1757 - acc: 0.995 - ETA: 8:38 - loss: 0.1756 - acc: 0.995 - ETA: 8:33 - loss: 0.1756 - acc: 0.995 - ETA: 8:28 - loss: 0.1755 - acc: 0.99